In [35]:
%matplotlib inline
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [36]:
# Load data
dataframe = pd.read_csv('data.csv')
# Remove unnecessary features
dataframe = dataframe.drop(['index', 'price', 'sq_price'], axis =1)
dataframe = dataframe[0:10]
dataframe

,area,bathrooms
0,2104.0,3.0
1,1600.0,3.0
2,2400.0,3.0
3,1416.0,2.0
4,3000.0,4.0
5,1985.0,4.0
6,1534.0,3.0
7,1427.0,3.0
8,1380.0,3.0
9,1494.0,3.0


In [37]:
# Add labels
dataframe.loc[:,('y1')] = [1,1,1,0,0,1,0,1,1,1]
dataframe.loc[:,('y2')] = dataframe['y1'] == 0
dataframe.loc[:,('y2')] = dataframe['y2'].astype(int)
dataframe

,area,bathrooms,y1,y2
0,2104.0,3.0,1,0
1,1600.0,3.0,1,0
2,2400.0,3.0,1,0
3,1416.0,2.0,0,1
4,3000.0,4.0,0,1
5,1985.0,4.0,1,0
6,1534.0,3.0,0,1
7,1427.0,3.0,1,0
8,1380.0,3.0,1,0
9,1494.0,3.0,1,0


In [38]:
# Prepare data for tensorflow
inputX = dataframe.loc[:,['area','bathrooms']].as_matrix()
inputY = dataframe.loc[:,['y1','y2']].as_matrix()
inputX,inputY

(array([[  2.10400000e+03,   3.00000000e+00],
        [  1.60000000e+03,   3.00000000e+00],
        [  2.40000000e+03,   3.00000000e+00],
        [  1.41600000e+03,   2.00000000e+00],
        [  3.00000000e+03,   4.00000000e+00],
        [  1.98500000e+03,   4.00000000e+00],
        [  1.53400000e+03,   3.00000000e+00],
        [  1.42700000e+03,   3.00000000e+00],
        [  1.38000000e+03,   3.00000000e+00],
        [  1.49400000e+03,   3.00000000e+00]]), array([[1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0]]))

In [60]:
# Hyperparameters
learning_rate = 0.000001
training_epochs = 2000
display_step = 50
n_samples = inputY.size
n_samples

20

In [61]:
# Create computation graph/Neural network
# None - Generic length of samples
# 2 - Number of features
x = tf.placeholder(tf.float32, [None,2])

# Create weights
W = tf.Variable(tf.zeros([2,2]))

# Add biases
b = tf.Variable(tf.zeros([2]))

# Multiply weights by inputs and add biases
y_values = tf.add(tf.matmul(x, W), b)

# Apply softmax
y = tf.nn.softmax(y_values)

# Feed in a matrix of labels
y_ = tf.placeholder(tf.float32, [None,2])

In [62]:
# Perform training
cost = tf.reduce_sum(tf.pow(y_ - y,2))/(2*n_samples) 

# Gradient Descent
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [63]:
# Initialize variables and tensorflow session

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [64]:
# Training loop
for i in range(training_epochs):
    sess.run(optimizer, feed_dict ={x:inputX, y_:inputY})
    
    # Write logs of training
    if i % display_step == 0:
        cc = sess.run(cost, feed_dict = {x: inputX, y_ :inputY})
        print "Training step:", '%04d' %(i), "Cost =", '{:.9f}'.format(cc)

print "Optimization finished"
training_cost = sess.run(cost, feed_dict = {x:inputX, y_:inputY})
print "Training cost =", training_cost, "W= ", sess.run(W), "b= ",sess.run(b),"\n"

Training step: 0000 Cost = 0.114958666
Training step: 0050 Cost = 0.109539941
Training step: 0100 Cost = 0.109539866
Training step: 0150 Cost = 0.109539807
Training step: 0200 Cost = 0.109539732
Training step: 0250 Cost = 0.109539673
Training step: 0300 Cost = 0.109539606
Training step: 0350 Cost = 0.109539531
Training step: 0400 Cost = 0.109539464
Training step: 0450 Cost = 0.109539405
Training step: 0500 Cost = 0.109539315
Training step: 0550 Cost = 0.109539248
Training step: 0600 Cost = 0.109539196
Training step: 0650 Cost = 0.109539129
Training step: 0700 Cost = 0.109539054
Training step: 0750 Cost = 0.109538987
Training step: 0800 Cost = 0.109538913
Training step: 0850 Cost = 0.109538853
Training step: 0900 Cost = 0.109538779
Training step: 0950 Cost = 0.109538712
Training step: 1000 Cost = 0.109538652
Training step: 1050 Cost = 0.109538577
Training step: 1100 Cost = 0.109538510
Training step: 1150 Cost = 0.109538436
Training step: 1200 Cost = 0.109538361
Training step: 1250 Cost 

In [65]:
sess.run(y, feed_dict = {x: inputX})

array([[ 0.71125221,  0.28874779],
       [ 0.66498977,  0.33501023],
       [ 0.73657656,  0.26342347],
       [ 0.64718789,  0.35281211],
       [ 0.78335613,  0.2166439 ],
       [ 0.70069474,  0.29930523],
       [ 0.65866327,  0.34133676],
       [ 0.64828628,  0.35171372],
       [ 0.64368278,  0.35631716],
       [ 0.65480113,  0.3451989 ]], dtype=float32)